참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

(`sympy` 1.0 이후에는 radian -> degree 변환 기능이 `mpmath` 로 독립)<br>
(For `sympy` 1.0 or later, `mpmath` is an external dependency)

In [ ]:
try:
    from sympy.mpmath import degrees as sy_degrees
except ImportError: 
    import sys
    from mpmath import degrees as sy_degrees

## 예제 03.002

### 문제에서 주어진 변수<br>Given Parameters

다음과 같은 변수들이 문제에서 주어졌다.

In [ ]:
d_al_m, d_st_m, T_Nm, G_st_Pa, G_al_Pa, L_al_m, L_st_m = sy.symbols('d_{al}[m], d_{st}[m], T[Nm], G_{st}[Pa], G_{al}[Pa], L_{al}[m], L_{st}[m]')

mm 단위 변수도 준비 해 둔다.

In [ ]:
d_al_mm = d_al_m * 1e3
d_st_mm = d_st_m * 1e3

변수값을 나중에 사용하기 위해 `dict` 형태로 저장해 둔다.

In [ ]:
subs_dict = {d_al_mm: 90,  # 알루미늄 부분 직경
             d_st_mm: 60,  # 철 부분 직경
             T_Nm: 80.0,  # 비틀림 하중
             G_st_Pa: 80e9,  # 철의 전단탄성계수
             G_al_Pa: 28e9,  # 알루미늄의 전단탄성계수
             L_al_m: 3,  # 알루미늄 부분 길이
             L_st_m: 1.5,  # 철 부분 길이
            }
subs_dict[d_al_m] = subs_dict[d_al_mm] * 1e-3 # 알루미늄 부분 직경
subs_dict[d_st_m] = subs_dict[d_st_mm] * 1e-3 # 철 부분 직경

### 1) 각 원통 A-B, B-C 에서의 최대 응력

#### 양 끝점에서의 반력

철 쪽의 반력과 알루미늄 쪽의 반력을 각각 아래와 같은 변수로 나타내자.

In [ ]:
T_st_Nm, T_al_Nm = sy.symbols('T_{st}[Nm], T_{al}[Nm]')

알루미늄 쪽의 반력과 철 쪽의 반력의 합과 비틀림 하중의 크기가 같을 것이다. 
$$T - T_{st} - T_{al} = 0 $$
python의 기호 연산 기능 으로는 다음과 같이 표시할 수 있다.

In [ ]:
eq_eq = sy.Eq(T_Nm - T_st_Nm - T_al_Nm, 0)

그러나 알루미늄 쪽의 반력과 철 쪽의 반력이 각기 어떤 값을 가질지는 정역학 statics 만으로는 알 수 없다.  이러한 문제를 부정정 statically indeterminate 문제라고 한다.  재료의 변형을 감안하면 각 반력도 알 수 있을 것이다.

비틀림 하중과 비틀림각 사이의 관계는 각각 다음과 같이 표시할 수 있다.

In [ ]:
J_st_m4 = sy.pi * d_st_m**4  / 32
theta_st_rad = T_st_Nm * L_st_m / (G_st_Pa * J_st_m4)

In [ ]:
J_al_m4 = sy.pi * d_al_m**4  / 32
theta_al_rad = T_al_Nm * L_al_m / (G_al_Pa * J_al_m4)

철 부분의 회전각과 알루미늄 부분의 회전각도 같아야 한다.
$$\theta_{st}=\theta_{al}$$

In [ ]:
geo_eq = sy.Eq(theta_st_rad, theta_al_rad)

degree 단위의 각도는 다음과 같이 나타낼 수 있을 것이다.

In [ ]:
# http://docs.sympy.org/0.7.1/modules/mpmath/functions/trigonometric.html
theta_st_deg =sy_degrees(theta_st_rad)
theta_al_deg = sy_degrees(theta_al_rad)

각 방정식은 다음과 같을 것이다.

In [ ]:
eq_eq

In [ ]:
geo_eq

연립 하여 풀면 다음과 같이 비틀림 하중을 얻을 수 있다.

In [ ]:
T_sol = sy.solve((eq_eq, geo_eq), (T_al_Nm, T_st_Nm))
T_sol

변수 값을 대입하면 다음과 같다.

In [ ]:
T_sol[T_al_Nm].subs(subs_dict)

In [ ]:
T_sol[T_st_Nm].subs(subs_dict)

아래는 프로그램이 맞게 작동했는지 확인하기 위해 추가된 부분이다. 

참고 : Francesco Mosconi, Travis + Anaconda + Jupyter, 2017 Aug 09, [Online] Available: https://github.com/ghego/travis_anaconda_jupyter.

In [ ]:
assert(1e-7 > na.norm(np.array([T_sol[T_al_Nm].subs(subs_dict), T_sol[T_st_Nm].subs(subs_dict)], dtype=float)
                      - np.array([37.5807787903894,42.4192212096106])))

#### 각 부 최대 전단 응력

최대 전단 응력은 각각 다음과 같이 구할 수 있다.
$$\tau_{max} = \frac{16T}{\pi d^3}$$

In [ ]:
tau_max_st = 16 * T_sol[T_st_Nm] / (sy.pi * d_st_m**3)
tau_max_st

In [ ]:
tau_max_al = 16 * T_sol[T_al_Nm] / (sy.pi * d_al_m**3)
tau_max_al

변수 값을 대입하면 다음과 같다.

In [ ]:
tau_st_Pa = float(tau_max_st.subs(subs_dict))
disp.display(disp.Math('$\\tau_{st} = %g(Pa)$' % tau_st_Pa))
disp.display(disp.Math('$\\tau_{st} = %g(MPa)$' % (tau_st_Pa*1e-6)))

In [ ]:
tau_al_Pa = float(tau_max_al.subs(subs_dict))
disp.display(disp.Math('$\\tau_{st} = %g(Pa)$' % tau_al_Pa))
disp.display(disp.Math('$\\tau_{st} = %g(MPa)$' % (tau_al_Pa*1e-6)))

아래는 프로그램이 맞게 작동했는지 확인하기 위해 추가된 부분이다. 

참고 : Francesco Mosconi, Travis + Anaconda + Jupyter, 2017 Aug 09, [Online] Available: https://github.com/ghego/travis_anaconda_jupyter.

In [ ]:
assert(1e-5 > abs(tau_al_Pa*1e-6 - 0.262548))

In [ ]:
assert(1e-5 > abs(tau_st_Pa*1e-6 - 1.00018))